In [7]:
%pip install torch==2.3.1+cpu torchvision==0.18.1+cpu -f https://download.pytorch.org/whl/torch_stable.html



^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install clip

Note: you may need to restart the kernel to use updated packages.


In [4]:
import clip
import torch
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

dataset_dir = "D:/clip-embedding/processed_samples"

def load_images(dataset_dir):
    images = []
    image_paths = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(dataset_dir, filename)
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
            images.append(image)
            image_paths.append(image_path)
    return images, image_paths

images, image_paths = load_images(dataset_dir)

with torch.no_grad():
    dataset_embeddings = torch.cat([model.encode_image(image) for image in images])
    dataset_embeddings /= dataset_embeddings.norm(dim=-1, keepdim=True)


In [9]:
def find_similar_images(input_image_path, dataset_embeddings, image_paths, top_k=5):

    input_image = preprocess(Image.open(input_image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        input_embedding = model.encode_image(input_image)
        input_embedding /= input_embedding.norm(dim=-1, keepdim=True)

    similarities = (input_embedding @ dataset_embeddings.T).squeeze(0)

    top_k_indices = similarities.topk(top_k).indices

    similar_image_paths = [image_paths[idx] for idx in top_k_indices]

    return similar_image_paths

input_image_path = r"D:/clip-embedding/processed_inputs/OOTD1.png"
similar_images = find_similar_images(input_image_path, dataset_embeddings, image_paths)

print("Most similar images:")
for img_path in similar_images:
    print(img_path)


Most similar images:
D:/clip-embedding/processed_samples\Screenshot 2024-07-28 012028.png
D:/clip-embedding/processed_samples\output_image1.png
D:/clip-embedding/processed_samples\Screenshot 2024-07-28 014514.png
D:/clip-embedding/processed_samples\Screenshot 2024-07-28 013849.png
D:/clip-embedding/processed_samples\Screenshot 2024-07-28 013530.png
